In [6]:
import pandas as pd
from utils import load_filtered_data

In [7]:
city = 'Sicklerville'
name = 'mixed'

# load city data
data = load_filtered_data(city)

In [8]:
# creating the nodes

nodes = data['user'].drop(columns=['friends'])
nodes = nodes.rename(columns={'user_id': 'id', 'name': 'people_name'})

nodes

id people_name  review_count        yelping_since  \
0     RgDVC3ZUBqpEe6Y1kPhIpw      Monica          1282  2009-02-23 14:08:31   
1     zkamNMEjihh3zN7lC7_WVw     Matthew            15  2006-02-04 15:53:46   
2     gVFxZMcuG_Tal2_TnpmUPg   Stephanie           146  2008-05-05 00:16:33   
3     g0_x4kVvJAYuk96oCcbOmw     Michael           455  2009-04-15 12:46:06   
4     -NbeVN5tnwdyYAvdNkKMjw     Dominic          1101  2012-04-25 19:31:00   
...                      ...         ...           ...                  ...   
3073  CUE13op55l9WZp6fBTUWnA      Nicole             2  2016-08-19 13:24:47   
3074  cACn6Qc1owO7hV_ZhcP74Q     Allison             1  2019-11-10 00:05:23   
3075  SpTYEhfInvALG1le1uu21w         Ray             2  2016-01-18 16:55:04   
3076  sGOCaCP7_SMlaRKPNO__CQ      Tahjae             2  2019-05-04 13:07:23   
3077  AUaVo2JAMwLGJmBo3M4VyQ      Nicole             1  2015-02-10 21:15:36   

      useful  funny   cool                                              elite  \
0      12640  10005  11149            2009,2010,2011,2012,2013,2014,2015,2016   
1         35      4     22                                                NaN   
2        347    100    124                      2011,2012,2013,2014,2015,2016   
3       1823    696    663  2009,2010,2011,2012,2013,2014,2015,2016,2017,2018   
4       6704   1959   4179  2012,2013,2014,2015,2016,2017,2018,2019,20,20,...   
...      ...    ...    ...                                                ...   
3073       0      0      0                                                NaN   
3074       0      0      0                                                NaN   
3075       5      0      0                                                NaN   
3076       0      0      0                                                NaN   
3077       0      0      0                                                NaN   

      fans  average_stars  ...  compliment_more  compliment_profile  \
0      804           4.17  ...              102                 108   
1        1           4.07  ...                1                   0   
2        6           3.68  ...               13                   1   
3       46           3.38  ...               11                   0   
4      193           3.75  ...               33                  18   
...    ...            ...  ...              ...                 ...   
3073     0           1.00  ...                0                   0   
3074     0           5.00  ...                0                   0   
3075     0           1.00  ...                0                   0   
3076     0           5.00  ...                0                   0   
3077     0           1.00  ...                0                   0   

      compliment_cute  compliment_list  compliment_note  compliment_plain  \
0                 138               65              898              2584   
1                   0                0                1                 3   
2                   0                0               10                 8   
3                   1                5               39                33   
4                   2                2              312               457   
...               ...              ...              ...               ...   
3073                0                0                0                 0   
3074                0                0                0                 0   
3075                0                0                0                 0   
3076                0                0                0                 0   
3077                0                0                0                 0   

      compliment_cool  compliment_funny  compliment_writer  compliment_photos  
0                1731              1731                551                177  
1                   2                 2                  0                  0  
2                  21                21                  8                  0  
3     

In [9]:
# creating the edges for users&bgt

temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}
connected = {}

for business in data['review']['business_id'].unique():
    reviews_of_business = data['review'][data['review']['business_id'] == business]
    users_that_reviewed_business = reviews_of_business['user_id'].unique()
    for i in range(len(users_that_reviewed_business)-1):
        for j in range(i+1, len(users_that_reviewed_business)):
            if users_that_reviewed_business[i] in connected:
                connected[users_that_reviewed_business[i]].append(users_that_reviewed_business[j])
            else:
                connected[users_that_reviewed_business[i]] = [users_that_reviewed_business[j]]
            if users_that_reviewed_business[j] in connected:
                connected[users_that_reviewed_business[j]].append(users_that_reviewed_business[i])
            else:
                connected[users_that_reviewed_business[j]] = [users_that_reviewed_business[i]]

# now do it for tip (data['tip'] has user_id and business_id)
for business in data['tip']['business_id'].unique():
    tip_of_business = data['tip'][data['tip']['business_id'] == business]
    users_that_tipped_business = tip_of_business['user_id'].unique()
    for i in range(len(users_that_tipped_business)-1):
        for j in range(i+1, len(users_that_tipped_business)):
            if users_that_tipped_business[i] in connected:
                connected[users_that_tipped_business[i]].append(users_that_tipped_business[j])
            else:
                connected[users_that_tipped_business[i]] = [users_that_tipped_business[j]]
            if users_that_tipped_business[j] in connected:
                connected[users_that_tipped_business[j]].append(users_that_tipped_business[i])
            else:
                connected[users_that_tipped_business[j]] = [users_that_tipped_business[i]]


THRESHOLD = 3

for business in data['review']['business_id'].unique():
    reviews_of_business = data['review'][data['review']['business_id'] == business]
    users_that_reviewed_business = reviews_of_business['user_id'].unique()
    for i in range(len(users_that_reviewed_business)-1):
        for j in range(i+1, len(users_that_reviewed_business)):
            if connected[users_that_reviewed_business[i]].count(users_that_reviewed_business[j]) >= THRESHOLD:
                # from i to j
                temp_edges_struct['from'].append(users_that_reviewed_business[i])
                temp_edges_struct['to'].append(users_that_reviewed_business[j])
                temp_edges_struct['weight'].append(connected[users_that_reviewed_business[i]].count(users_that_reviewed_business[j]))
                temp_edges_struct['type'].append('reviewed-same-restaurant')
                # from j to i
                temp_edges_struct['from'].append(users_that_reviewed_business[j])
                temp_edges_struct['to'].append(users_that_reviewed_business[i])
                temp_edges_struct['weight'].append(connected[users_that_reviewed_business[j]].count(users_that_reviewed_business[i]))
                temp_edges_struct['type'].append('reviewed-same-restaurant')
            

for business in data['tip']['business_id'].unique():
    tip_of_business = data['tip'][data['tip']['business_id'] == business]
    users_that_tipped_business = tip_of_business['user_id'].unique()
    for i in range(len(users_that_tipped_business)-1):
        for j in range(i+1, len(users_that_tipped_business)):
            if connected[users_that_tipped_business[i]].count(users_that_tipped_business[j]) >= THRESHOLD:
                # from i to j
                temp_edges_struct['from'].append(users_that_tipped_business[i])
                temp_edges_struct['to'].append(users_that_tipped_business[j])
                temp_edges_struct['weight'].append(connected[users_that_tipped_business[i]].count(users_that_tipped_business[j]))
                temp_edges_struct['type'].append('tipped-same-restaurant')
                # from j to i
                temp_edges_struct['from'].append(users_that_tipped_business[j])
                temp_edges_struct['to'].append(users_that_tipped_business[i])
                temp_edges_struct['weight'].append(connected[users_that_tipped_business[j]].count(users_that_tipped_business[i]))
                temp_edges_struct['type'].append('tipped-same-restaurant')

edges_usersbgt = pd.DataFrame(temp_edges_struct)

# remove repeated lines
edges_usersbgt = edges_usersbgt.drop_duplicates()

# now sum the weights of the repeated
edges_usersbgt = edges_usersbgt.groupby(['from', 'to', 'type']).agg({'weight': 'sum'}).reset_index()


# adding the edges for friendships

user_ids_set = set(data['user']['user_id'])
temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}

for index, row in data['user'].iterrows():
    user_id = row['user_id']
    friends = row['friends']
    if pd.notnull(friends):
        friend_list = friends.split(', ')
        for friend in friend_list:
            friendship_pair = tuple(sorted([user_id, friend]))
            if friendship_pair[0] in user_ids_set and friendship_pair[1] in user_ids_set:
                temp_edges_struct['from'].append(friendship_pair[0])
                temp_edges_struct['to'].append(friendship_pair[1])
                temp_edges_struct['weight'].append(2)
                temp_edges_struct['type'].append('friendship')

edges_friendships = pd.DataFrame(temp_edges_struct)
edges_friendships = edges_friendships.drop_duplicates()

edges = pd.concat([edges_usersbgt, edges_friendships])

def multiply_values(x):
    result = 1
    for value in x:
        result *= value
    return result

edges = edges.groupby(['from', 'to']).agg({'weight': multiply_values}).reset_index()

edges = edges.drop_duplicates()

edges['type'] = 'reviewed-same-restaurant-and-friendship'

edges

from                      to  weight  \
0     -0MIp6WKJ8QvGnYZQ5ETyg  BmVwbsL8l0imz4slonyMaA       2   
1     -6MrOJCN-eIe37-LT-uPoQ  BmVwbsL8l0imz4slonyMaA       2   
2     -6MrOJCN-eIe37-LT-uPoQ  PfkF7k5bk9ftSXrC8Q7GaQ       3   
3     -6MrOJCN-eIe37-LT-uPoQ  ePxjx8tee06mn5h87X-Sxg       3   
4     -6MrOJCN-eIe37-LT-uPoQ  i-v015Oa8uK4JiHe0GC8mQ       3   
...                      ...                     ...     ...   
2552  zkamNMEjihh3zN7lC7_WVw  lnQyKu1NZTMThnUYGh6jgg       3   
2553  zkamNMEjihh3zN7lC7_WVw  ruTAZ7_Q7YAeEMeIoun6AQ       3   
2554  zkamNMEjihh3zN7lC7_WVw  ulRdB25JWcztx_n6N67yog       4   
2555  zvYSqlpOr2Nhi6ct_91vYw  2qfEiGZrOYwm2HX9ai4SNA       9   
2556  zvYSqlpOr2Nhi6ct_91vYw  tBlxhHAjHPY2AdMEonmylw       3   

                                         type  
0     reviewed-same-restaurant-and-friendship  
1     reviewed-same-restaurant-and-friendship  
2     reviewed-same-restaurant-and-friendship  
3     reviewed-same-restaurant-and-friendship  
4     reviewed-same-restaurant-and-friendship  
...                                       ...  
2552  reviewed-same-restaurant-and-friendship  
2553  reviewed-same-restaurant-and-friendship  
2554  reviewed-same-restaurant-and-friendship  
2555  reviewed-same-restaurant-and-friendship  
2556  reviewed-same-restaurant-and-friendship  

[2557 rows x 4 columns]

In [10]:
# save data
nodes.to_csv(f'nodes_and_edges/{city}_{name}_nodes.csv', index=False)
edges.to_csv(f'nodes_and_edges/{city}_{name}_edges.csv', index=False)